In [ ]:
import os
import re
import pandas as pd
from selenium import webdriver

In [ ]:
# Credentials
USERNAME = os.environ.get('GCCAMPUS_USERNAME')
PASSWORD = os.environ.get('GCCAMPUS_PASSWORD')

# Bool for scraping FR descriptions
FRENCH = False

In [ ]:
browser = webdriver.Chrome()

In [ ]:
# Navigate to GCcampus and login
if FRENCH:
    main_url = 'https://idp.csps-efpc.gc.ca/idp/login-fr.jsp'
else:
    main_url = 'https://idp.csps-efpc.gc.ca/idp/Authn/UserPassword'

browser.get(main_url)
browser.find_element_by_id('j_username').send_keys(USERNAME)
browser.find_element_by_id('j_password').send_keys(PASSWORD)
browser.find_element_by_id('cbPrivacy').click()
browser.find_element_by_xpath("//button[@type='submit']").click()

In [ ]:
%%time
# Loop through catalogue and get all links to courses
if FRENCH:
    list_url = 'https://learn-apprendre.csps-efpc.gc.ca/application/fr/courses-solr?page='
else:
    list_url = 'https://learn-apprendre.csps-efpc.gc.ca/application/en/courses-solr?page='

course_links = []
for i in range(42): # Limit of ?page=41 in EN (therefore use range(42)); ?page=41 in FR
    browser.get(list_url + str(i))
    mars = browser.find_elements_by_css_selector('.field-items a')
    for elem in mars:
        course_links.append(elem.get_attribute('href'))

In [ ]:
# Compile regex to extract course codes
regex = re.compile(pattern=r'[a-zA-Z]{1}\d{3}(?:\s{1}-\s{1}MODULE\s{1}\d{1})?')

In [ ]:
# Dict to map problematic / exceptional course codes
exception_dict = {
    'H200': 'H200_MODULE 1'
}

In [ ]:
%%time
# For each link in 'course_links', navigate to page, grab course description (HTML
# tags included), search for course code, and save to 'desc_dict'
desc_dict = {}
for i, link in enumerate(course_links):
    print('{0} / {1}'.format(i, len(course_links)))
    browser.get(link)
    # Grab description
    desc = browser.find_elements_by_css_selector('.field-item[property="content:encoded"]')[0].get_attribute('innerHTML')
    # Grab title and extract course code
    title = browser.find_elements_by_css_selector('.page-title')[0].get_attribute('innerHTML')
    title_search = regex.findall(title)
    pkey = title_search[0] if title_search else link
    if pkey in exception_dict:
        pkey = exception_dict[pkey]
    desc_dict[pkey] = desc

In [ ]:
# Store 'desc_dict' in DataFrame for processing
df = pd.DataFrame.from_dict(desc_dict, orient='index')
df.reset_index(level=0, inplace=True)
df.columns = ['course_code', 'desc']

In [ ]:
# Transform relative links to absolute links
df['desc'] = df['desc'].str.replace('href="/application/en/',
                                    'href="https://learn-apprendre.csps-efpc.gc.ca/application/en/',
                                    regex=False)
df['desc'] = df['desc'].str.replace('href="/application/en/',
                                    'href="https://learn-apprendre.csps-efpc.gc.ca/application/fr/',
                                    regex=False)

In [ ]:
# Remove junk info
df['desc'] = df['desc'].str.replace(' This link will open in a new window', '', regex=False)
df['desc'] = df['desc'].str.replace(' Ce lien va ouvrir dans une nouvelle fenêtre', '', regex=False)

In [ ]:
# Replace new line characters '\r' and '\n' with a space
df['desc'] = df['desc'].str.replace('\r', ' ', regex=False)
df['desc'] = df['desc'].str.replace('\n', ' ', regex=False)

In [ ]:
# Remove all double+ spaces
df['desc'] = df['desc'].str.replace(r'  +', ' ', regex=True)

In [ ]:
# Export to CSV
df.to_csv('scraped_{0}.csv'.format('fr' if FRENCH else 'en'), sep=',', encoding='utf-8')